In [1]:
pip install transformers datasets torch nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset
dataset= load_dataset("cnn_dailymail","3.0.0")
article= dataset['train'][0]['article']
print("original article: \n",article)

original article: 
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office c

In [6]:
from transformers import BartForConditionalGeneration, BartTokenizer

tokenizer= BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model= BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

In [10]:
article= dataset["test"][0]["article"]
reference_summary= dataset["test"][0]["highlights"]

inputs= tokenizer(article,return_tensors= "pt",truncation=True, max_length= 1024)
summary_ids= model.generate(inputs["input_ids"],num_beams=4,max_length=100,early_stopping=True)
generated_summary= tokenizer.decode(summary_ids[0],skip_special_tokens=True)
print("reference summary: \n",reference_summary,"\n")
print("generated summary: \n",generated_summary)


reference summary: 
 Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis . 

generated summary: 
 The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki said it was a move toward greater justice.


In [11]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2014ec4e15cc17232a7e36efdfbf078898e88983ab0d4f9d496f419379214f78
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [15]:
from rouge_score import rouge_scorer
scorer= rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"],use_stemmer=True)
scores= scorer.score(reference_summary,generated_summary)
print("rouge scores")
for key,value in scores.items():
  print(f"{key}: Precision: {value.precision:.4f},Recall:{value.recall:.4f}, F1-score: {value.fmeasure:.4f}")

rouge scores
rouge1: Precision: 0.3654,Recall:0.5588, F1-score: 0.4419
rouge2: Precision: 0.2549,Recall:0.3939, F1-score: 0.3095
rougeL: Precision: 0.3269,Recall:0.5000, F1-score: 0.3953


In [16]:
pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00


In [20]:
generated_summary = ["Candidate 1 summary", "Candidate 2 summary"]  # Example generated summaries
reference_summary = ["Reference 1 summary", "Reference 2 summary"]  # Corresponding reference summaries

# Ensure both lists have the same number of elements
assert len(generated_summary) == len(reference_summary), "Different number of candidates and references"

from bert_score import score

P, R, F1 = score(generated_summary, reference_summary, lang="en", model_type="bert-base-uncased")

print("\nBERT scores")
print(f"Precision: {P.mean():.2f}, Recall: {R.mean():.2f}, F1-score: {F1.mean():.2f}")



BERT scores
Precision: 0.83, Recall: 0.83, F1-score: 0.83


In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name= "t5-base"
tokenizer= T5Tokenizer.from_pretrained(model_name)
model= T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [22]:
text= """The T5 model is a transformer-based machine learning model developed by Google Research. It is trained on a diverse dataset and performs multiple NLP tasks bu [reventing them into text-to-text problems]"""

input_text= "summarize: "+ text
input_ids= tokenizer.encode(input_text,return_tensors="pt",max_length=512,truncation=True)

In [23]:
summary_ids= model.generate(input_ids, max_length=50,num_beams=5,length_penalty=0.2,early_stopping=True)
summary= tokenizer.decode(summary_ids[0],skip_special_tokens=True)
print("summary: \n",summary)

summary: 
 the T5 model is a transformer-based machine learning model developed by google research . it performs multiple NLP tasks bu [reventing them into text-to-text problems]


In [24]:
from datasets import load_dataset
dataset= load_dataset("cnn_dailymail","3.0.0")

train_data= dataset["train"]
val_data= dataset["validation"]
test_data= dataset["test"]